<a href="https://colab.research.google.com/github/vitthal-bhandari/Homophobia-Transphobia-Detection/blob/master/Baselines/English_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initializing environment

In [1]:
#check gpu usage
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr  6 07:58:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#check ram usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install datasets
!pip install transformers
!sudo apt-get install git-lfs
!pip3 install torch==1.10.2+cu102 torchvision==0.11.3+cu102 torchaudio===0.10.2+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html

     |████████████████████████████████| 325 kB 9.5 MB/s 
     |████████████████████████████████| 1.1 MB 82.6 MB/s 
     |████████████████████████████████| 136 kB 66.4 MB/s 
     |████████████████████████████████| 212 kB 96.5 MB/s 
     |████████████████████████████████| 77 kB 8.0 MB/s 
     |████████████████████████████████| 127 kB 74.0 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 64.8 MB/s 
     |████████████████████████████████| 144 kB 93.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 3.8 MB 7.9 MB/s 


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset
import tensorflow as tf
import pandas as pd
import numpy as np
import torch

In [5]:
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


# Reading pre-processed data

In [8]:
eng_train_dataset = load_dataset("csv", data_files="eng_augmented_data.csv")
eng_train = eng_train_dataset["train"]

eng_dev_dataset = load_dataset("csv", data_files="eng_dev.csv")
eng_dev = eng_dev_dataset["train"]

eng_test_dataset = load_dataset("csv", data_files="eng_test.csv")
eng_test = eng_test_dataset["train"]

print(len(eng_train), len(eng_dev), len(eng_test))

Using custom data configuration default-d409d2e4734d1b50


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d409d2e4734d1b50/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-b01d124b1c4ce04b


...

Using custom data configuration default-a1faf62149fdc32a


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a1faf62149fdc32a/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

4530 791 990


# Declaring model name

In [10]:
model_ckpt = "bert-base-uncased"

# Tokenization

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [12]:
# Define tokenizer function

def tokenize(batch):
  return tokenizer(batch["text"], padding=True, truncation=True)

In [13]:
# Ensuring there are no empty records so that tokenization doesn't fail

eng_train = eng_train.filter(lambda example: example['text'] is not None)
eng_dev = eng_dev.filter(lambda example: example['text'] is not None)
eng_test = eng_test.filter(lambda example: example['text'] is not None)

print( len(eng_train), len(eng_dev), len(eng_test) )

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

4530 791 990


In [14]:
# Apply tokenizer across all splits in the corpus

eng_train_encoded = eng_train.map(tokenize, batched=True, batch_size=None)
eng_dev_encoded = eng_dev.map(tokenize, batched=True, batch_size=None)
eng_test_encoded = eng_test.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
print(eng_train_encoded.column_names)

['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask']


# Feature Extraction

In [16]:
from transformers import AutoModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

print(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


cuda


In [17]:
def extract_hidden_states(batch):
  
  #Place model inputs on GPU
  inputs = {k:v.to(device) for k, v in batch.items() 
  if k in tokenizer.model_input_names}

  #Extract last hidden states
  with torch.no_grad():
    last_hidden_state = model(**inputs).last_hidden_state
  
  #Return vector for [CLS] token
  return {"hidden_state": last_hidden_state[:, 0].cpu().numpy()}

In [18]:
eng_train_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
eng_dev_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])
eng_test_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [19]:
eng_train_hidden = eng_train_encoded.map(extract_hidden_states, batched=True)
eng_dev_hidden = eng_dev_encoded.map(extract_hidden_states, batched=True)
eng_test_hidden = eng_test_encoded.map(extract_hidden_states, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
eng_train_hidden.column_names

['label',
 'text',
 'input_ids',
 'token_type_ids',
 'attention_mask',
 'hidden_state']

In [22]:
X_eng_train = np.array(eng_train_hidden['hidden_state'])
X_eng_dev = np.array(eng_dev_hidden['hidden_state'])
X_eng_test = np.array(eng_test_hidden['hidden_state'])

Y_eng_train = np.array(eng_train_hidden['label'])
Y_eng_dev = np.array(eng_dev_hidden['label'])
Y_eng_test = np.array(eng_test_hidden['label'])

print( X_eng_train.shape, X_eng_dev.shape, X_eng_test.shape, Y_eng_train.shape, Y_eng_dev.shape, Y_eng_test.shape)

(4530, 768) (791, 768) (990, 768) (4530,) (791,) (990,)


# Training an LR classifier

In [23]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(max_iter = 3000)
lr_clf.fit(X_eng_train, Y_eng_train)

LogisticRegression(max_iter=3000)

In [24]:
lr_clf.score(X_eng_dev, Y_eng_dev)

0.8407079646017699

In [46]:
#Evaluating a dummy classifier for comparison which always returns the majority class

from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_eng_train, Y_eng_train)
dummy_clf.score(X_eng_dev, Y_eng_dev)

0.07332490518331226

In [47]:
Y_eng_preds = lr_clf.predict(X_eng_dev)

In [48]:
from sklearn.metrics import classification_report

target_names = ['homophobic', 'Non-anti LGBTQIA+ content', 'transphobic']
print(classification_report(Y_eng_dev, Y_eng_preds, target_names=target_names))

                           precision    recall  f1-score   support

               homophobic       0.24      0.53      0.34        58
Non-anti LGBTQIA+ content       0.96      0.87      0.91       731
              transphobic       0.00      0.00      0.00         2

                 accuracy                           0.84       791
                macro avg       0.40      0.47      0.42       791
             weighted avg       0.90      0.84      0.87       791

